# Model Family Performance Comparison: Comprehensive Heatmap Analysis

This notebook generates comprehensive performance comparison heatmaps across different the model families (CLIP, DINOv2, ViT) and sizes (Small, Base, Large). It processes pre-computed performance tables and creates styled visualizations to compare experimental configurations.

This notebook loads the results created by `create_tables_full_model_performances.ipynb` and generates performance tables for each model family on all task and some probe type combinations.
These are the tables of Appendix Figure 5. 

The analysis loads CSV performance tables for each model and creates color-coded heatmaps showing test balanced accuracy across datasets and experimental setups (different layer combinations and pooling strategies).

**Output**: Multi-model heatmaps with distinct color schemes for each model family, displaying performance metrics across datasets. Each model within a family uses a different colormap, enabling easy comparison of how model size affects performance across various experimental configurations and datasets.

In [ ]:
import sys
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import matplotlib.colors as mcolors

sys.path.append("..")
sys.path.append("../..")

from helper import init_plotting_params
from constants import base_model_name_mapping, BASE_PATH_PROJECT, FOLDER_SUBSTRING

In [3]:
init_plotting_params()

{
  "agg.path.chunksize": 0,
  "axes.labelsize": 13.0,
  "axes.titlesize": 14.0,
  "axes3d.trackballsize": 0.667,
  "boxplot.flierprops.markersize": 6.0,
  "boxplot.meanprops.markersize": 6.0,
  "errorbar.capsize": 0.0,
  "figure.figsize": [
    6.4,
    4.8
  ],
  "figure.labelsize": "large",
  "figure.titlesize": "large",
  "font.cursive": [
    "Apple Chancery",
    "Textile",
    "Zapf Chancery",
    "Sand",
    "Script MT",
    "Felipa",
    "Comic Neue",
    "Comic Sans MS",
    "cursive"
  ],
  "font.family": [
    "sans-serif"
  ],
  "font.fantasy": [
    "Chicago",
    "Charcoal",
    "Impact",
    "Western",
    "xkcd script",
    "fantasy"
  ],
  "font.monospace": [
    "DejaVu Sans Mono",
    "Bitstream Vera Sans Mono",
    "Computer Modern Typewriter",
    "Andale Mono",
    "Nimbus Mono L",
    "Courier New",
    "Courier",
    "Fixed",
    "Terminal",
    "monospace"
  ],
  "font.sans-serif": [
    "DejaVu Sans",
    "Bitstream Vera Sans",
    "Computer Modern Sans Serif

In [ ]:
base_res_path = BASE_PATH_PROJECT / f"results_{FOLDER_SUBSTRING}_rebuttal/plots/per_model_all_performances"

In [5]:
color_maps = [
    'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
    ]
model_name_mapping = {
    "mae-vit-base-p16_v2": "MAE-B-16",
    "mae-vit-large-p16_v2": "MAE-L-16",
    "mae-vit-base-p16": "MAE-B-16",
    "mae-vit-large-p16": "MAE-L-16",
    
}
exp_name_mapping = {
    'CLS last layer': "Last layer\n(CLS,\nlinear)",
    'AP last layer': "Last layer\n(AP,\nlinear)",
    'CLS+AP last layer (linear)': "Last layer\n(CLS+AP,\nlinear)",
    'CLS+AP layers from middle & last block (linear)': "Two layers\n(CLS+AP,\nlinear)",
    'CLS+AP layers from middle & last blocks (linear)': "Two layers\n(CLS+AP,\nlinear)",
    'CLS+AP layers from quarterly block (linear)': "Four layers\n(CLS+AP,\nlinear)",
    'CLS+AP layers from quarterly blocks (linear)': "Four layers\n(CLS+AP,\nlinear)",
    'CLS+AP layers from all blocks (linear)': "All layers\n(CLS+AP,\nlinear)",
    'CLS+AP last layer (attentive)': "Last layer\n(CLS+AP,\nattentive)",
    'CLS+AP layers from middle & last block (attentive)': "Two layers\n(CLS+AP,\nattentive)",
    'CLS+AP layers from middle & last blocks (attentive)': "Two layers\n(CLS+AP,\nattentive)",
    'CLS+AP layers from quarterly block (attentive)': "Four layers\n(CLS+AP,\nattentive)",
    'CLS+AP layers from quarterly blocks (attentive)': "Four layers\n(CLS+AP,\nattentive)",
    'CLS+AP layers from all blocks (attentive)': "All layers\n(CLS+AP,\nattentive)",
    'All tokens last layer (attentive)': "Last layer\n(all tokens,\nattentive)",
}

In [6]:
from collections import defaultdict
model_path_dict = defaultdict(list)
for i, path in enumerate(base_res_path.rglob("*.csv")):
    model_name = path.stem.split('perf_table_')[-1]
    if "_v2" in model_name:
        continue
    if 'dinov2-' in model_name:
        model_path_dict['dinov2'].append(path)
    elif 'mae-' in model_name: 
        model_path_dict['mae'].append(path)
    elif 'vit_' in model_name: 
        model_path_dict['vit'].append(path)
    else:
        model_path_dict['clip'].append(path)

In [7]:
df_list = {}
for model_family, models_paths in model_path_dict.items():
    for i, path in enumerate(models_paths):
        model_name = base_model_name_mapping[path.stem.split("perf_table_")[1]]
    
        df = pd.read_csv(path, index_col=0, header=1)
        df.index.name = None
        df = df.drop(index='dataset_fmt')
        df.rename(index={'Diabetic Retinopathy': 'Diabetic\nRetinopathy',
                         "PASCAL VOC 2007": "PASCAL VOC07"}, inplace=True)
        df.columns = [exp_name_mapping[col] for col in df.columns]
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df = df.dropna(axis=1, how='all')
    
        # add model name as meta column
        df.insert(0, "Model", model_name)
    
        df_list[model_name] = df
df_list.keys()

dict_keys(['ViT-L-16', 'ViT-S-16', 'ViT-B-16', 'CLIP-B-32', 'CLIP-B-16', 'CLIP-L-14', 'MAE-B-16', 'MAE-L-16', 'DINOv2-L-14', 'DINOv2-S-14', 'DINOv2-B-14'])

In [8]:
tmp = pd.concat([df.loc['mean perf. gain'] for name, df in df_list.items() if 'MAE' not in name ], axis=1).T
tmp.drop(columns=['Model']).mean().sort_values()

Last layer\n(CLS,\nlinear)                   0.0
Last layer\n(CLS+AP,\nlinear)           1.305768
Last layer\n(CLS+AP,\nattentive)        1.842278
Two layers\n(CLS+AP,\nlinear)           2.583626
Four layers\n(CLS+AP,\nlinear)          3.291752
Two layers\n(CLS+AP,\nattentive)         3.68103
All layers\n(CLS+AP,\nlinear)           3.796811
Four layers\n(CLS+AP,\nattentive)       4.816793
Last layer\n(all tokens,\nattentive)    5.309449
All layers\n(CLS+AP,\nattentive)        5.538655
dtype: object

In [9]:
model_lists_to_consider = [
    ['ViT-S-16', 'ViT-B-16', 'ViT-L-16'],
    ['CLIP-B-32', 'CLIP-B-16', 'CLIP-L-14',],
    ['MAE-B-16', 'MAE-L-16', ],
    ['DINOv2-S-14', 'DINOv2-B-14', 'DINOv2-L-14']
]

# Remove 'Model' column from each df and create a multi-level column index
drop_col_1 = ['Last layer\n(CLS+AP,\nlinear)','Two layers\n(CLS+AP,\nlinear)','Four layers\n(CLS+AP,\nlinear)', 'Last layer\n(CLS+AP,\nattentive)']
drop_col_2 = ['Last layer\n(CLS+AP,\nlinear)','Last layer\n(CLS+AP,\nattentive)',]


def style_multimodel_heatmap(df, model_cmaps, precision=2):
    """
    df: pandas DataFrame with MultiIndex columns (level 0 = model, level 1 = metric)
    model_cmaps: dict of {model_name: colormap_name}
    """
    models = df.columns.get_level_values(0).unique()

    def highlight_row(row):
        styles = []
        for model in models:
            cmap_name = model_cmaps.get(model, "Reds")
            cmap = cm.get_cmap(cmap_name)

            # Extract only this row for the model
            values = row[model].values.astype(float)
            
            # Normalize per row (row-wise within model)
            vmin, vmax = values.min(), values.max()
            if vmin == vmax:  # avoid divide-by-zero
                normed = [0.5] * len(values)
            else:
                normed = (values - vmin) / (vmax - vmin)
            
            for n in normed:
                rgba = cmap(n)
                color = mcolors.to_hex(rgba)
                styles.append(f"background-color: {color}")
        return styles

    styler = df.style.format(precision=precision, na_rep="")
    styler = styler.apply(highlight_row, axis=1)
    # Apply background_gradient per model block
    for model in df.columns.get_level_values(0).unique():
        #print(model)
        cmap_name = model_cmaps.get(model, "Reds")
        # select all columns under this model
        subset = [col for col in df.columns if col[0] == model]
        styler = styler.background_gradient(cmap=cmap_name, axis=1, subset=subset)
    return styler

    

def get_combined_df(models_to_consider):
    dfs_multi = []
    for model in models_to_consider:
        df = df_list[model]
        if model not in models_to_consider:
            continue
        df_copy = df.drop(columns="Model").copy()
        if model == 'CLIP-B-32':
            df_copy = df_copy.drop(columns=['Two layers\n(CLS+AP,\nattentive)','Four layers\n(CLS+AP,\nattentive)'])
        try:
            df_copy = df_copy.drop(columns=drop_col_1)
        except:
            df_copy = df_copy.drop(columns=drop_col_2)
        df_copy.columns = pd.MultiIndex.from_product([[model], df_copy.columns])
        dfs_multi.append(df_copy)
    combined_df = pd.concat(dfs_multi, axis=1)
    return combined_df


def style_combined(combined_df):
    color_maps = {
        'ViT-S-16': "Reds",
        'ViT-B-16': "Greens", 
        'ViT-L-16': "Blues",
        'CLIP-B-32': "Reds",
        'CLIP-B-16': "Greens", 
        'CLIP-L-14': "Blues", 
        'MAE-B-16': "Greens", 
        'MAE-L-16': "Blues", 
        'DINOv2-S-14': "Reds",
        'DINOv2-B-14': "Greens",
        'DINOv2-L-14': "Blues",
    }
    
    styled = style_multimodel_heatmap(combined_df, model_cmaps=color_maps)
    styled = styled.set_table_styles([
        # top-level column headers (models)
        {'selector': 'th.col_heading.level0', 'props': [('text-align', 'center')]},
        # second-level column headers (metrics)
        {'selector': 'th.col_heading.level1', 'props': [('text-align', 'center')]},
        # row headers (dataset names)
        {'selector': '.row_heading', 'props': [('text-align', 'left')]}
    ])
    col_width="75px"
    data_font_size="15px"
    header_font_size="13px"
    index_font_size="13px"
    return styled.set_table_styles([
            {'selector': 'table', 'props': [('table-layout', 'fixed')]},
            
            # Data cells - largest font
            {'selector': 'td:not(.row_heading)', 'props': [
                ('width', col_width),
                ('text-align', 'center'),
                ('font-size', data_font_size),
                ('font-weight', 'bold'),
                ('white-space', 'nowrap')
            ]},
            
            {'selector': 'th.col_heading', 'props': [
                ('width', col_width),
                ('font-size', header_font_size),
                ('vertical-align', 'top'),
                ('text-align', 'center'),
            ]},
            
            {'selector': '.row_heading', 'props': [
                ('font-weight', 'bold'),
                ('font-size', index_font_size),
                ('width', 'auto'),
                ('text-align', 'left'),
            ]}
        ])



for models_to_consider in model_lists_to_consider:
    combined_df = get_combined_df(models_to_consider)
    display(style_combined(combined_df))
    



/tmp/ipykernel_3698751/861197447.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)


/tmp/ipykernel_3698751/861197447.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)


/tmp/ipykernel_3698751/861197447.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)


/tmp/ipykernel_3698751/861197447.py:24: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(cmap_name)
